# Code Style

### Что хотим на выходе?
- модуль/библиотека на Python
- plain sql и некоторые специфичные конструкции


### Что в основе?
- open source библиотека sqlparse
- самостоятельная реализация правил


### Коротко о результатах
Link: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf

- Количество правил, реализация которых взята в работу: 23

- Количество реализованных правил: 21 (~90%)

- Количество новых кейсов, которые добавлены в общий перечень правил и  взяты в работу: 4 *(or in join, with stmnt, window functions, teradata funcs: create (multiset volatile) table)*


### Как использовано время?

- Анализ существующих библиотек, доступных в открытом доступе, собранных на python, alive, чтение кода, сопоставление правил.    ~ 40%

- Написание кода для основной части правил (80% для mvp).                                                                        ~ 30%

- Написание кода для новых кейсов, логов, тестов и отладка ошибок (плюс правила до 90%).                                                        ~ 30%

### Чему научились к этому времени?
- Примеры

In [2]:
from SQLParser import *

In [3]:
sql = HandlerSQL()
raw = '''
        select msc.* 
            , t.start_dttm 
            , t.end_dttm , t.country_code
            , datediff(t.end_dttm, t.start_dttm) as true_days_diff
            , case
                when t.country_code != 'RU' then 'not RU'
                else t.country_code
            end as rus_or_not
            , case
                when t.start_dttm < to_timestamp('2022-10-01 00:00:00') then to_timestamp('2022-10-01 00:00:00')
                else t.start_dttm
            end as start_dt
            , case when t.end_dttm > to_timestamp('2022-10-31 23:59:59') then to_timestamp('2022-10-31 23:59:59')
                else t.end_dttm
            end as end_dt
        from roaming.roaming_report as t
        join prd2_dic_v.geo_country as cntr 
        on t.country_code = substr(cntr.country_code, 1, 2)
        join cls_moscow as msc on t.subs_id = msc.subs_id
        where t.start_dttm <= '2022-10-31 23:59:59'
            and t.end_date >= '20221001'
'''

In [4]:
sql.run(raw)


        CodeStyle Warning in column_name_rule_warn(): Если в запросе участвуют несколько таблиц, то при указании наименования столбца необходимо явно указывать 
        какой таблице принадлежит столбец. Проверьте строки ['CASE', 'END AS rus_or_not ,', 'CASE', 'END AS start_dt ,', 'CASE', 'END AS end_dt']. (SQL Style Guide. Подзапросы. Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf)
        

        CodeStyle Warning in row_max_symbols_warn(): Максимальное количество символов в строке – не более 80. (SQL Style Guide. Общие конструкции. 
        Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf). 
        Перечень строк с ошибками сохранен в объекте row_max_symbols_df_global.
        



SELECT msc.* ,
       t.start_dttm ,
       t.end_dttm,
       t.country_code ,
       DATEDIFF(t.end_dttm, t.start_dttm) AS true_days_diff ,
       CASE
           WHEN t.country_code != 'RU' THEN 'not RU'
           ELSE t.country_code
       END AS rus_or_not ,
       CASE
           WHEN t.start_dttm < to_timestamp('2022-10-01 00:00:00') THEN to_timestamp('2022-10-01 00:00:00')
           ELSE t.start_dttm
       END AS start_dt ,
       CASE
           WHEN t.end_dttm > to_timestamp('2022-10-31 23:59:59') THEN to_timestamp('2022-10-31 23:59:59')
           ELSE t.end_dttm
       END AS end_dt
FROM roaming.roaming_report AS t
JOIN prd2_dic_v.geo_country AS cntr ON t.country_code = SUBSTR(cntr.country_code, 1, 2)
JOIN cls_moscow AS msc ON t.subs_id = msc.subs_id
WHERE 1 = 1
    AND t.start_dttm <= '2022-10-31 23:59:59'
    AND t.end_date >= '20221001'


In [ ]:
sql.row_max_symbols_df

In [5]:
sql = HandlerSQL()
raw = '''
        select
            subs_id
            , end_date
            , start_dttm
            , end_dttm
            , time_per_trip
            , dense_rank() over (partition by subs_id, end_date order by time_per_trip desc ) as rank_in_day
            , temporary_fdsfsf
            , sum(total_days_per_month) over (order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_3
        from user_time_in_trip
'''

In [6]:
sql.run(raw)


        CodeStyle Warning in row_max_symbols_warn(): Максимальное количество символов в строке – не более 80. (SQL Style Guide. Общие конструкции. 
        Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf). 
        Перечень строк с ошибками сохранен в объекте row_max_symbols_df_global.
        



SELECT subs_id ,
       end_date ,
       start_dttm ,
       end_dttm ,
       time_per_trip ,
       DENSE_RANK() OVER (PARTITION BY subs_id,
                    end_date
               ORDER BY time_per_trip DESC) AS rank_in_day ,
                    temporary_fdsfsf ,
       SUM(total_days_per_month) OVER (
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_3
FROM user_time_in_trip


In [9]:
sql = HandlerSQL()
raw = '''
select
            t.year_of_calendar
            , t.calendar_date
            , t.end_of_month
            , usr.subs_id
            , usr.msisdn
            , usr.start_dttm
            , usr.end_dttm
            , usr.home_province_code
            , usr.province_code_correct
            , usr.country_code
            , usr.home_flag
            , usr.end_date
            , usr.home_region
            , usr.destination
            , case
                when (unix_timestamp(usr.end_dttm) - unix_timestamp(t.calendar_date)) > 86400 then 86400 / 60 / 60
                when (unix_timestamp(usr.end_dttm) - unix_timestamp(t.calendar_date)) <= 86400 and (unix_timestamp(t.calendar_date) - unix_timestamp(usr.start_dttm)) <= 86400 then
                    round((unix_timestamp(usr.end_dttm) - unix_timestamp(usr.start_dttm)) / 60 / 60, 2)
                when (unix_timestamp(usr.end_dttm) - unix_timestamp(t.calendar_date)) <= 86400 and (unix_timestamp(t.calendar_date) - unix_timestamp(usr.start_dttm)) > 86400 then
                    round((unix_timestamp(usr.end_dttm) - unix_timestamp(t.calendar_date)) / 60 / 60, 2)
            end as time_per_day
            , datediff(usr.end_dttm, usr.start_dttm) as days_per_trip
            , case
                when non_flick.connections_per_day is not null then 1
                else 0
            end as flick_flag
            , case
                when lt.rank_in_month is not null then 1
                else 0
            end as longest_trip_flag
        from month as t
        join non_flickers as non_flick on usr.subs_id = non_flick.subs_id
                and t.calendar_date = non_flick.calendar_date
                and connections_per_day is not null
'''

In [10]:
sql.run(raw)


        CodeStyle Warning in row_max_symbols_warn(): Максимальное количество символов в строке – не более 80. (SQL Style Guide. Общие конструкции. 
        Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf). 
        Перечень строк с ошибками сохранен в объекте row_max_symbols_df_global.
        



SELECT t.year_of_calendar ,
       t.calendar_date ,
       t.end_of_month ,
       usr.subs_id ,
       usr.msisdn ,
       usr.start_dttm ,
       usr.end_dttm ,
       usr.home_province_code ,
       usr.province_code_correct ,
       usr.country_code ,
       usr.home_flag ,
       usr.end_date ,
       usr.home_region ,
       usr.destination ,
       CASE
           WHEN (UNIX_TIMESTAMP(usr.end_dttm) - UNIX_TIMESTAMP(t.calendar_date)) > 86400 THEN 86400 / 60 / 60
           WHEN (UNIX_TIMESTAMP(usr.end_dttm) - UNIX_TIMESTAMP(t.calendar_date)) <= 86400
                AND (UNIX_TIMESTAMP(t.calendar_date) - UNIX_TIMESTAMP(usr.start_dttm)) <= 86400 THEN ROUND((UNIX_TIMESTAMP(usr.end_dttm) - UNIX_TIMESTAMP(usr.start_dttm)) / 60 / 60, 2)
           WHEN (UNIX_TIMESTAMP(usr.end_dttm) - UNIX_TIMESTAMP(t.calendar_date)) <= 86400
                AND (UNIX_TIMESTAMP(t.calendar_date) - UNIX_TIMESTAMP(usr.start_dttm)) > 86400 THEN ROUND((UNIX_TIMESTAMP(usr.end_dttm) - UNIX_TIMESTAMP(t.calen

In [ ]:
len('           WHEN (UNIX_TIMESTAMP(usr.end_dttm) - UNIX_TIMESTAMP(t.calendar_date)) <= 86400')

In [11]:
sql = HandlerSQL()
raw = '''
        select
            t.*
            , sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d
            , sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 3 preceding and current row) as cum_sum_120d
            , sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 4 preceding and current row) as cum_sum_150d
            , sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 5 preceding and current row) as cum_sum_180d
            , sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 6 preceding and current row) as cum_sum_total
        from time_per_month_spent_by_users_in_loc as t
'''

In [12]:
sql.run(raw)


        CodeStyle Warning in row_max_symbols_warn(): Максимальное количество символов в строке – не более 80. (SQL Style Guide. Общие конструкции. 
        Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf). 
        Перечень строк с ошибками сохранен в объекте row_max_symbols_df_global.
        



SELECT t.* ,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d ,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 3 preceding AND CURRENT ROW) AS cum_sum_120d ,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 4 preceding AND CURRENT ROW) AS cum_sum_150d ,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 5 preceding AND CURRENT ROW) AS cum_sum_180d ,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 6 preceding AND CURRENT ROW) AS cum_sum_total
FROM time_per_month_spent_by_users_in_loc AS t


In [13]:
sql = HandlerSQL()
raw = '''

CREATE MULTISET TABLE PRD_DM.MC_FORECAST_RETURN_REVENUE 
    (
    REPORT_DATE DATE Format 'YY/MM/DD',
    RETURN_CURRENT_MONTH_DATE DECIMAL(18,6),
    RETURN_CURRENT_MONTH_LAST_YEAR_DATE DECIMAL(18,6),
    RETURN_CURRENT_MONTH_LAST_YEAR DECIMAL(18,6),
    RETURN_CURRENT_YEAR DECIMAL(18,6),
    RETURN_LAST_YEAR_LAST_DATE DECIMAL(18,6),
    RETURN_LAST_YEAR DECIMAL(18,6),
    REVENUE_CURRENT_MONTH_DATE DECIMAL(18,6),
    REVENUE_CURRENT_MONTH_LAST_YEAR_DATE DECIMAL(18,6),
    REVENUE_CURRENT_MONTH_LAST_YEAR DECIMAL(18,6),
    REVENUE_CURRENT_YEAR DECIMAL(18,6),
    REVENUE_LAST_YEAR_LAST_DATE DECIMAL(18,6),
    REVENUE_LAST_YEAR DECIMAL(18,6)
	)
PRIMARY INDEX ( REPORT_DATE );

INSERT INTO PRD_DM.MC_FORECAST_RETURN_REVENUE
SELECT * FROM UAT_DM.MC_FORECAST_RETURN_REVENUE;

'''

In [14]:
sql.run(raw)


CREATE multiset TABLE prd_dm.mc_forecast_return_revenue (
    report_date DATE format 'YY/MM/DD',
    return_current_month_date decimal(18, 6),
    return_current_month_last_year_date decimal(18, 6),
    return_current_month_last_year decimal(18, 6),
    return_current_year decimal(18, 6),
    return_last_year_last_date decimal(18, 6),
    return_last_year decimal(18, 6),
    revenue_current_month_date decimal(18, 6),
    revenue_current_month_last_year_date decimal(18, 6),
    revenue_current_month_last_year decimal(18, 6),
    revenue_current_year decimal(18, 6),
    revenue_last_year_last_date decimal(18, 6),
    revenue_last_year decimal(18, 6)
    )
PRIMARY INDEX (report_date);


INSERT INTO prd_dm.mc_forecast_return_revenue
SELECT *
FROM uat_dm.mc_forecast_return_revenue;


In [15]:
sql = HandlerSQL()
raw = '''

CREATE MULTISET TABLE PRD_DM.MC_FORECAST_RETURN_REVENUE 
    (
    REPORT_DATE DATE Format 'YY/MM/DD',
    RETURN_CURRENT_MONTH_DATE DECIMAL(18,6),
    RETURN_CURRENT_MONTH_LAST_YEAR_DATE DECIMAL(18,6),
    RETURN_CURRENT_MONTH_LAST_YEAR DECIMAL(18,6),
    RETURN_CURRENT_YEAR DECIMAL(18,6),
    RETURN_LAST_YEAR_LAST_DATE DECIMAL(18,6),
    RETURN_LAST_YEAR DECIMAL(18,6),
    REVENUE_CURRENT_MONTH_DATE DECIMAL(18,6),
    REVENUE_CURRENT_MONTH_LAST_YEAR_DATE DECIMAL(18,6),
    REVENUE_CURRENT_MONTH_LAST_YEAR DECIMAL(18,6),
    REVENUE_CURRENT_YEAR DECIMAL(18,6),
    REVENUE_LAST_YEAR_LAST_DATE DECIMAL(18,6),
    REVENUE_LAST_YEAR DECIMAL(18,6)
	)
PRIMARY INDEX ( REPORT_DATE );

INSERT INTO PRD_DM.MC_FORECAST_RETURN_REVENUE
SELECT * FROM UAT_DM.MC_FORECAST_RETURN_REVENUE;

CREATE MULTISET TABLE amount_per_day_usage
     (
      first_day_of_month DATE Format 'YY/MM/DD',
      REPORT_DATE DATE Format 'YY/MM/DD',
      SUBS_ID DECIMAL(12,0),
      BSEGMENT VARCHAR(20) ,
      rated_amount_sum DECIMAL(18,6)
	  )
PRIMARY INDEX ( SUBS_ID )
ON COMMIT PRESERVE ROWS;

 CREATE MULTISET VOLATILE TABLE amount_per_day_usage
     (
      first_day_of_month DATE Format 'YY/MM/DD',
      REPORT_DATE DATE Format 'YY/MM/DD',
      SUBS_ID DECIMAL(12,0),
      BSEGMENT VARCHAR(20) CHARACTER SET Unicode CaseSpecific,
      rated_amount_sum DECIMAL(18,6)
	  )
PRIMARY INDEX ( SUBS_ID )
ON COMMIT PRESERVE ROWS;

'''

In [16]:
sql.run(raw)


CREATE multiset TABLE prd_dm.mc_forecast_return_revenue (
    report_date DATE format 'YY/MM/DD',
    return_current_month_date decimal(18, 6),
    return_current_month_last_year_date decimal(18, 6),
    return_current_month_last_year decimal(18, 6),
    return_current_year decimal(18, 6),
    return_last_year_last_date decimal(18, 6),
    return_last_year decimal(18, 6),
    revenue_current_month_date decimal(18, 6),
    revenue_current_month_last_year_date decimal(18, 6),
    revenue_current_month_last_year decimal(18, 6),
    revenue_current_year decimal(18, 6),
    revenue_last_year_last_date decimal(18, 6),
    revenue_last_year decimal(18, 6)
    )
PRIMARY INDEX (report_date);


INSERT INTO prd_dm.mc_forecast_return_revenue
SELECT *
FROM uat_dm.mc_forecast_return_revenue;


CREATE multiset TABLE amount_per_day_usage (
    first_day_of_month DATE format 'YY/MM/DD',
    report_date DATE format 'YY/MM/DD',
    subs_id decimal(12, 0),
    bsegment varchar(20),
    rated_amount_sum d

In [17]:
sql = HandlerSQL()
raw = '''
SELect
                             t.report_date
  ,      t.gendeR
,   active,sUm(t.vol) as sum_vol, Count(t.xsdf) as dfv
, avg(sdf) as fpl
,   sum(total_days_per_month) over (partition by subs_id, id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_1

, buzz
, sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_2
  ,  t.gendeR
, sum(total_days_per_month) over (order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_3
,       foo
,   foo1

, count(total_days_per_month) over (partition by id order by subs_id rows between 2 preceding and current row) as cum_sum_90d_1
, sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_2
, sum(total_days_per_month) over (partition by subs_id, id, a, b, c order by subs_id, end_of_month, a, b, x rows between 2 preceding and current row) as cum_sum_90d_3
, sum(total_days_per_month) over (order by subs_id) as cum_sum_90d_4
, rank() over (partition by subs_id, id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_5


FROM active_data_users AS t
LEFT JOIN traffic AS trf ON
JOIN table as tab on t.subs_id = tab.subs_id or t.report_date = tab.report_date1_
and (t.subs_id = trf.subs_id or t.id = trf.id)
left join bar on t.num = bar.num and t.dt = bar.dt and (t.foo = tab.foo or t.buzz = tab.buzz)
where  t> 3  and  t=0 and (d>t.wef or d>6 or t.vol = 49 or t.vol = 10 or y =13) and f=  12 or t.vol  =   r.top and i in   (1111, 111222, 2222222222, 22222222444444444, 4444444444444555555555555555555)
GROUP BY
1, 2, 3,4
, t.report_date
, age_group
, t.gender;

select * from tab join buzz on a=b where 1=1 and id != 111 or name in (xyz, abc);
select * from tabl full outer join foo as f on name  =name where t = 9;

select t.var 
from foo as f
join buzz as b
    on (
        f.id=b.id
        and f.dt=b.dt
        and xyz.sdfjhfd = c.asdnweionf
    )
where 1=1 
    and (
    b.date >= '2022-10-01' 
    and t.name != 'not in RU'
    )
'''

In [18]:
sql.run(raw)


        CodeStyle Warning in column_name_rule_warn(): Если в запросе участвуют несколько таблиц, то при указании наименования столбца необходимо явно указывать 
        какой таблице принадлежит столбец. Проверьте строки ['active,', 'AVG(sdf) AS fpl,', 'SUM(total_days_per_month) OVER (PARTITION BY subs_id,', 'id', 'ORDER BY subs_id,', 'end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_1 ,', 'buzz,', 'SUM(total_days_per_month) OVER (PARTITION BY subs_id', 'ORDER BY subs_id,', 'end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_2 ,', 'SUM(total_days_per_month) OVER (', 'ORDER BY subs_id,', 'end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_3,', 'foo,', 'foo1 ,', 'COUNT(total_days_per_month) OVER (PARTITION BY id', 'ORDER BY subs_id ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_1,', 'SUM(total_days_per_month) OVER (PARTITION BY subs_id', 'ORDER BY subs_id,', 'end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cu


SELECT t.report_date ,
       t.gender,
       active,
       SUM(t.vol) AS sum_vol,
       COUNT(t.xsdf) AS dfv,
       AVG(sdf) AS fpl,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id,
                    id
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_1 ,
       buzz,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_2 ,
       t.gender,
       SUM(total_days_per_month) OVER (
               ORDER BY subs_id,
                    end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_3,
       foo,
       foo1 ,
       COUNT(total_days_per_month) OVER (PARTITION BY id
               ORDER BY subs_id ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_1,
       SUM(total_days_per_month) OVER (PARTITION BY subs_id
               ORDER BY subs_id,

### Что нужно доработать
- Join'ы (некоторые кейсы с составными ключами)
- Create table (несколько подряд идущих)
- WITH statement
- Последовательные сложные запросы
- Процедуры

### Что планируется успеть до конца января?

- Базовый сценарий: реализация всех взятых в работу правил + новых кейсов + логирование

- Оптимистичный сценарий: расширение функционала специфичными функциями sql (teradata), вывод модуля во Flask

In [19]:
raw1 = '''
SELECT
		t.subs_id
		, t.start_dttm
		, t.start_date
		, t.end_dttm
		, t.a_number
		, t.b_number
	FROM PRD2_DDS_V.USAGE AS t
	WHERE t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
		AND t.traffic_type_id = 2
		AND (t.b_number = '7757' OR t.b_number = 11)
		AND (t.a_number IN ('312', '7757') OR Length(t.a_number) = 11 OR t.b_number1 = 11)
        AND (t.b_number1 = '7757' OR t.b_number1 = 11 OR t.b_number1 = 12)

'''
sql = HandlerSQL()
sql.run(raw1)


        CodeStyle Warning in or_in_a_row_warn(): Найдено 3 подряд идущих условий OR. Рекомендуется замена условий на выражение IN (x, y, z). 
        (SQL Style Guide. WHERE/ON. Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf)
        



SELECT t.subs_id ,
       t.start_dttm ,
       t.start_date ,
       t.end_dttm ,
       t.a_number ,
       t.b_number
FROM prd2_dds_v.usage AS t
WHERE 1 = 1
    AND t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
    AND t.traffic_type_id = 2
    AND (t.b_number = '7757'
         OR t.b_number = 11)
    AND (t.a_number IN ('312',
                        '7757')
         OR LENGTH(t.a_number) = 11
         OR t.b_number1 = 11)
    AND (t.b_number1 = '7757'
         OR t.b_number1 = 11
         OR t.b_number1 = 12)


In [20]:
raw1 = '''
WITH ALL_USERS_MC AS (
	SELECT
		t.subs_id
		, t.msisdn
		, t.transaction_id
		, t.transaction_dttm
		, t.amount_rub
		, t.revenue_rub
		, t.update_ts
		, serv.valid_from_date
		, serv.valid_to_date
	FROM prd2_dds_v2.mobile_commerce AS t
	JOIN prd2_dds_v2.service_instance AS serv ON t.subs_id = serv.subs_id
		AND serv.service_id = 30709
	WHERE t.transaction_dttm > '2022-09-30 00:00:00' and t.dt > 1 
),

ORDERED_MOVINGS AS (
	SELECT
		t.subs_id
		, t.start_dttm
		, t.start_date
		, t.end_dttm
		, t.a_number
		, t.b_number
	FROM PRD2_DDS_V.USAGE AS t
	WHERE t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
		AND t.traffic_type_id = 2
		AND (t.b_number = '7757' OR t.b_number = 11)
		AND (t.a_number IN ('312', '7757') OR Length(t.a_number) = 11 OR t.b_number1 = 11)
        AND (t.b_number1 = '7757' OR t.b_number1 = 11 OR t.b_number1 = 12))
,

tempo as (
select t.subs_id, t.a_number, t.foo, t.buzz
from buka as t
where t.foo is not null
),

acdc as (
select t.foo1, t.buzz1
from tempo as t

)
select * from ORDERED_MOVINGS
'''
sql = HandlerSQL()
sql.run(raw1)


        CodeStyle Warning in or_in_a_row_warn(): Найдено 3 подряд идущих условий OR. Рекомендуется замена условий на выражение IN (x, y, z). 
        (SQL Style Guide. WHERE/ON. Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf)
        


WITH all_users_mc AS (
    SELECT t.subs_id,
           t.msisdn,
           t.transaction_id,
           t.transaction_dttm,
           t.amount_rub,
           t.revenue_rub,
           t.update_ts,
           serv.valid_from_date,
           serv.valid_to_date
    FROM prd2_dds_v2.mobile_commerce AS t
    JOIN prd2_dds_v2.service_instance AS serv 
        ON (
        t.subs_id = serv.subs_id
        AND serv.service_id = 30709
        )
    WHERE 1 = 1
        AND t.transaction_dttm > '2022-09-30 00:00:00'
        AND t.dt > 1
),

ordered_movings AS (
    SELECT t.subs_id,
           t.start_dttm,
           t.start_date,
           t.end_dttm,
           t.a_number,
           t.b_number
    FROM prd2_dds_v.usage AS t
    WHERE 1 = 1
        AND t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
        AND t.traffic_type_id = 2
        AND (t.b_number = '7757'
             OR t.b_number = 11)
        AND (t.a_number IN ('312',
                            '7757')
         

In [21]:
raw1 = '''
WITH ORDERED_MOVINGS AS (
	SELECT
		t.subs_id
		, t.start_dttm
		, t.start_date
		, t.end_dttm
		, t.a_number
		, t.b_number
	FROM PRD2_DDS_V.USAGE AS t
	WHERE t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
		AND t.traffic_type_id = 2
		AND (t.b_number = '7757' OR t.b_number = 11)
		AND (t.a_number IN ('312', '7757') OR Length(t.a_number) = 11)
        AND (t.b_number1 = '7757' OR t.b_number1 = 11)
        )

select * from ORDERED_MOVINGS
'''
sql = HandlerSQL()
sql.run(raw1)


        CodeStyle Warning in or_in_a_row_warn(): Найдено 1 подряд идущих условий OR. Рекомендуется замена условий на выражение IN (x, y, z). 
        (SQL Style Guide. WHERE/ON. Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf)
        


WITH ordered_movings AS (
    SELECT t.subs_id,
           t.start_dttm,
           t.start_date,
           t.end_dttm,
           t.a_number,
           t.b_number
    FROM prd2_dds_v.usage AS t
    WHERE 1 = 1
        AND t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
        AND t.traffic_type_id = 2
        AND (t.b_number = '7757'
             OR t.b_number = 11)
        AND (t.a_number IN ('312',
                            '7757')
             OR LENGTH(t.a_number) = 11)
        AND (t.b_number1 = '7757'
        OR t.b_number1 = 11
)


SELECT *
FROM ordered_movings
